* meta-algorithm : 한사람의 의견을 듣기보다 여러사람의 의견을 들어 종합해보자 

* classification imbalance : target 값이 true 인것과 false 인것의 데이터의 격차가 큰 경우 문제


# 7.1 Classifiers using multiple samples of the dataset

* AdaBoost 
    - 장점 : 오류율이 낮고 코드가 쉬움, 가장 좋은 분류기를 가지고 분류, 조절을 위한 매개변수가 없음
    - 단점 : Outliers 에 민감
    - 활용 : 수치형 값, 명목형 값
    
* 앙상블 메소드 (ensemble method 또는 meta algorithm) : 서로다른 알고리즘 형태로 사용, 설정다르게 또는 데이터 집합의 서로 다른 부분을 분류기에 적용


## 7.1.1 Building classifiers from randomly resampled data: bagging

* **bagging** : 다양한 분류기를 통합하는 방법, 원본 데이터 집합으로부터 S 개의 새로운 데이터 집합을 만듬 (새로운 데이터 집합의 크기는 원본 데이터 집합의 크기와 같음) 

S 번 만들어진 각각의 데이터 집합은 원본을 대신하기 위해, 원본으로부터 임의로 선택된 예제들로 구축, 두번 이상 같은 예제를 선택할 수도 있고 새로운 집합에 없을 수도 있다.  

S 개의 분류기를 새로운 데이터에 적용하고 다수결로 결정 

* 랜덤포레스트 : 여러 개의 결정 트리들을 임의적으로 학습하는 방식의 앙상블 방법, 랜덤 포레스트 방법은 크게 다수의 결정 트리를 구성하는 학습 단계와 입력 벡터가 들어왔을 때, 분류하거나 예측하는 테스트 단계로 구성되어있다. 랜덤 포레스트는 검출, 분류, 그리고 회귀 등 다양한 애플리케이션으로 활용 by wikipedia

## 7.1.2 Boosting

* **boosting** : 배깅은 언제나 동일한 유형의 분류기를 사용하는데 비해 부스팅은 순차적으로 다른 유형의 분류기를 사용, 이전 분류기에서 잘못 분류된 데이터에 초점을 맞춰 새로운 분류기를 만듬 

* AdaBoost 접근법
    - 준비 : 모든 유형의 데이터 처리, 약한 학습기 사용
    - 학습 : 동일한 데이터 집합으로 약한 학습기를 여러번 훈련시킴 
    - 검사 : 오류율
    - 사용 : 두가지 분류 항목중 하나를 예측, 2 개 이상의 데이터 분류에는 같은 방버을 여러번 적용해야 함 
    
    
# 7.2 Train: improving the classifier by focusing on errors
    
* 분류기 
    - 약함: 분류기가 임의로 추축하는 것보다 일을 잘 처리 (분류 항복이 두개인 경우 오류율이 50% 보다 높다.) 
    - 강함: 낮은 오류율 

* AdaBoost (adaptive boosting) : 가중치는 훈련 데이터에 있는 모든 예제에 적용 
초기에는 가중치들은 모두 동일, 훈련데이터로 훈련 후 오류를 계산 분류기는 동일한 데이터 집합을 가지고 두 번째 훈련을 시도함. 두번쨰 분류기의 훈련은 훈련 집합의 가중치를 적응하게 됨. 

첫번째에서 이미 분류된 예제들은 가중치를 낮게 부여하고 확실하게 분류되지 못한 예제들은 더 높은 가중치를 부여함 

adaboost 는 모든 분류기에 a(알파) 값을 부여해 오류를 기반으로 답을 구함 

$e = \frac{잘못분류된데이터}{전체데이터}$
    
$a= \frac{1}{2} ln( \frac{1-e}{e} )$

a 를 계산 후, 가중치 벡터 d 를 갱신 

* 확실하게 예측한 경우 

D(t+1) = $\frac{D(t) e^{-a}}{Sum(D)}$

* 예측하지 못한 경우 

D(t+1) = $\frac{D(t) e^{a}}{Sum(D)}$

D 가 계단되고 나면 AdaBoost 는 반복함 훈련 오류가 0이 되거나, 분류기들의 수가 사용자가 정한 값에 도달할 때까지 훈련과 가중치 조절을 되풀이함 weight-adjusting


# 7.3 의사결정 스텀프로 약한 학습기 생성


